# Diplomatura de Especialización en Desarrollo de Aplicaciones con Inteligencia Artificial - Inteligencia Artificial para Juegos (Game AI) - Sesión 2 (Ejemplo)

<font color='orange'>Entorno de Maze 2D para búsqueda de rutas entre dos puntos de un mapa</font>

El objetivo de este notebook es comparar métodos de busqueda ciega y busqueda informada A\*. El ejemplo propuesto es un problema para buscar rutas y así planear las acciones requeridas para el desplazamiento desde un punto a otro en un mapa 2D. La clase MazeSearchProblem necesita ser completada (métodos *actions* y *result*). Deberá implementarse 2 heurísticas para A* (straightline_dist:distancia en linea recta, y manhatan_dist:distancia Manhatan). Despues de implementar ello pruebe las implementaciones con el laberinto `mapa_sesion2.txt`. Al final de este notebook se encuentran algunas preguntas que usted debe contestar.

**Únicamente se requiere completar código en las zonas indicadas (es decir, en: ## TO DO: Completar)**


## Definición de entornos

### Clase <b>Maze</b>
La clase Maze es representada por una matriz (grid) de espacios, los cuales pueden estar libres o bloqueados. Cada celda del grid puede tener uno de los siguientes carateres:

    '#': Indica una celda con obstaculo  (impasable) 
    '~': Indica una celda con agua (pasable, con costo 5)
    '=': Indica una celda con arena (pasable, con costo 3)
    ' ': Indica una celda vacia (pasable con costo 1)
    'S': Indica la celda de inicio (Start)
    'E': Indica la celda de salida ( End )

In [0]:
class Maze:

    def __init__(self, grid):
        """ Construye el maze a partir del grid ingresado
            grid: debe ser una matriz (lista de listas), ejemplo [['#','S',' '],['#',' ','E']]  """
        self.grid = grid
        self.numRows = len(grid)
        self.numCols = len(grid[0])
        for i in range(self.numRows):
            for j in range(self.numCols):
                if len(grid[i]) != self.numCols:
                    raise "Grid no es Rectangular"
                if grid[i][j] == 'S':
                    self.startCell = (i,j)
                if grid[i][j] == 'E':
                    self.exitCell= (i,j)
        if self.exitCell == None:
            raise "No hay celda de Inicio"
        if self.startCell == None:
            raise "No hay celda de salida"
   
    def isPassable(self, row, col):
        """ Retorna true si la celda (row,col) es pasable  (' ' o '~' o '=') """
        return self.isWater(row, col) or self.isSand(row,col) or self.isClear(row, col)    
   
    def isWater(self,row,col):
      return self.grid[row][col] == '~'
    
    def isSand(self,row,col):
      return self.grid[row][col] == '='
    
    def isClear(self, row, col):
        """ Retorna true si la celda (row,col) esta vacia  (' ') """
        return self.grid[row][col] == ' '
    
    def isBlocked(self, row,col):
        """ Retorna true si la celda (row,col) tiene obstaculo ('#') """
        return self.grid[row][col] == '#'   
        
    def getNumRows(self):
        """ Retorna el numero de filas en el maze """
        return self.numRows
  
    def getNumCols(self):
        """ Retorna el numero de columnas en el maze """
        return self.numCols  
   
    def getStartCell(self):
        """ Retorna la posicion (row,col) de la celda de inicio """
        return self.startCell
  
    def getExitCell(self):
        """ Retorna la posicion (row,col) de la celda de salida """
        return self.exitCell

    def __getAsciiString(self):
        """ Retorna el string de vizualizacion del maze """
        lines = []
        headerLine = ' ' + ('-' * (self.numCols)) + ' '
        lines.append(headerLine)
        for row in self.grid:
            rowLine = '|' + ''.join(row) + '|'
            lines.append(rowLine)
        lines.append(headerLine)
        return '\n'.join(lines)

    def __str__(self):
        return self.__getAsciiString()

### Funcion para cargar una laberinto de archivo de disco

In [0]:
def readMazeFromFile(file):
    """ Lee un archivo que contiene un laberinto y retorna una instancia de Maze con dicho laberinto"""
    fin = open(file)
    lines = fin.read().splitlines()
    grid = []
    for line in lines: 
        grid.append(list(line))
    return Maze(grid)

## Algoritmos de Búsqueda

### Clase <b>SearchProblem</b>

Esta es una clase abstracta para definir problemas de busqueda. Se debe hacer subclases que implementen los metodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de busqueda.

In [0]:
class SearchProblem(object):
    def __init__(self, initial, goal=None):
        """Este constructor especifica el estado inicial y posiblemente el estado(s) objetivo(s),
        La subclase puede añadir mas argumentos."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones que pueden ser ejecutadas en el estado dado.
        El resultado es tipicamente una lista."""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada en el estado state.
        La accion debe ser alguna de self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True si el estado pasado satisface el objetivo."""
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con 
        la accion action, asumiendo un costo c para llegar hasta state1. 
        El metodo por defecto cuesta 1 para cada paso en el camino."""
        return c + 1

### Clase <b>Node</b>

Estructura de datos para almacenar la informacion de un nodo en un <b>arbol de busqueda</b>. Contiene información del nodo padre y el estado que representa el nodo. Tambien incluye la accion que nos llevo al presente nodo y el costo total del camino desde el nodo raiz hasta este nodo.

In [0]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Crea un nodo de arbol de busqueda, derivado del nodo parent y accion action"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        "Devuelve los nodos alcanzables en un paso a partir de este nodo."
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next = problem.result(self.state, action)
        return Node(next, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next))

    def solution(self):
        "Retorna la secuencia de acciones para ir de la raiz a este nodo."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Retorna una lista de nodos formando un camino de la raiz a este nodo."
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def __lt__(self, node):
        return self.state < node.state
    
    def __eq__(self, other): 
        "Este metodo se ejecuta cuando se compara nodos. Devuelve True cuando los estados son iguales"
        return isinstance(other, Node) and self.state == other.state
    
    def __repr__(self):
        return "<Node {}>".format(self.state)
    
    def __hash__(self):
        return hash(self.state)

### <b> Frontera tipo cola FIFO (first-in first out) para BFS</b> 

In [0]:
from collections import deque

class FIFOQueue(deque):
    """Una cola First-In-First-Out"""
    def pop(self):
        return self.popleft()

### <b> Frontera tipo cola de prioridad ordenada por una funcion de costo (para best_first_graph_search y A*)</b> 

In [0]:
import heapq
class FrontierPQ:
    "Una Frontera ordenada por una funcion de costo (Priority Queue)"
    
    def __init__(self, initial, costfn=lambda node: node.path_cost):
        "Inicializa la Frontera con un nodo inicial y una funcion de costo especificada (por defecto es el costo de camino)."
        self.heap   = []
        self.states = {}
        self.costfn = costfn
        self.add(initial)
    
    def add(self, node):
        "Agrega un nodo a la frontera."
        cost = self.costfn(node)
        heapq.heappush(self.heap, (cost, node))
        self.states[node.state] = node
        
    def pop(self):
        "Remueve y retorna el nodo con minimo costo."
        (cost, node) = heapq.heappop(self.heap)
        self.states.pop(node.state, None) # remove state
        return node
    
    def replace(self, node):
        "node reemplaza al nodo de la Fontera que tiene el mismo estado que node."
        if node.state not in self:
            raise ValueError('{} no tiene nada que reemplazar'.format(node.state))
        for (i, (cost, old_node)) in enumerate(self.heap):
            if old_node.state == node.state:
                self.heap[i] = (self.costfn(node), node)
                heapq._siftdown(self.heap, 0, i)
                return

    def __contains__(self, state): return state in self.states
    
    def __len__(self): return len(self.heap)

### <b>Algoritmo general de búsqueda con memoria de nodos expandidos (Graph Search)</b>

Algoritmo de general de busqueda ciega con memoria de estados visitados. El argumento frontier debe ser una cola vacia. Si la frontera es tipo FIFO hace busqueda en amplitud (BFS), si la frontera es una pila hará busqueda en profundidad (DFS)

In [0]:
def graph_search(problem, frontier):
    frontier.append(Node(problem.initial))
    explored = set()     # memoria de estados visitados
    visited_nodes = []   # almacena nodos visitados durante la busqueda
    while frontier:
        node = frontier.pop()
        visited_nodes.append(node)
        if problem.goal_test(node.state):
            return node, visited_nodes
        explored.add(node.state)
        
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None

### <b> Algoritmo Best-First-Graph-Search </b> 
Algoritmo general de busqueda con información. La frontera es una cola de prioridad ordenada por la funcion de evaluacion f 

In [0]:
def best_first_graph_search(problem, f):
    """Busca el objetivo expandiendo el nodo de la frontera con el menor valor de la funcion f. Memoriza estados visitados
    Antes de llamar a este algoritmo hay que especificar La funcion f(node). Si f es node.depth tenemos Busqueda en Amplitud; 
    si f es node.path_cost tenemos Busqueda  de Costo Uniforme. Si f es una heurística tenemos Busqueda Voraz;
    Si f es node.path_cost + heuristica(node) tenemos A* """

    frontier = FrontierPQ( Node(problem.initial), f )  # frontera tipo cola de prioridad ordenada por f
    explored = set()     # memoria de estados visitados
    visited_nodes = []   # almacena nodos visitados durante la busqueda
    while frontier:
        node = frontier.pop()
        visited_nodes.append(node)        
        if problem.goal_test(node.state):
            return node, visited_nodes
        explored.add(node.state)
        for action in problem.actions(node.state):
            child = node.child_node(problem, action)
            if child.state not in explored and child.state not in frontier:
                frontier.add(child)
            elif child.state in frontier:
                incumbent = frontier.states[child.state] 
                if f(child) < f(incumbent):
                    frontier.replace(child)

### <b> Algoritmo A* </b> 
A* es un caso especial de best_first_graph_search con f = path_cost + heuristic

In [0]:
def astar_search(problem, heuristic):
    f = lambda node: node.path_cost + heuristic(node, problem)
    return best_first_graph_search(problem, f)

def nullheuristic(node, problem):   
    return 0

### <b> Heurísticas para A* </b> 
Se debe implementar las heurísticas abajo para A* 

In [0]:
#Para elevar un número a una potencia, se sugiere usar la libreria math
import math
math.pow(4,2)

16.0

In [0]:
def straightline_dist(node, problem):
    "Distancia en linea recta desde la celda de node hasta la celda Objetivo (problem.goal)"
    ## TODO: Completar

    ##################

def manhatan_dist(node, problem):
    "Distancia Manhatan (o city block) desde la celda de node hasta la celda Objetivo (problem.goal)"
    ## TODO: Completar

    ##################


In [0]:
import math
def straightline_dist(node, problem):
    "Distancia en linea recta desde la celda de node hasta la celda Objetivo (problem.goal)"
    delta_x = node.state[0] - problem.goal[0]
    delta_y = node.state[1] - problem.goal[1]
    return math.sqrt( math.pow(delta_x, 2) + math.pow(delta_y, 2) )

def manhatan_dist(node, problem):
    "Distancia Manhatan (o city block) desde la celda de node hasta la celda Objetivo (problem.goal)"
    delta_x = node.state[0] - problem.goal[0]
    delta_y = node.state[1] - problem.goal[1]
    return  abs(delta_x) + abs(delta_y) 

## Clases para el entorno maze 2D

###  <b> Clase MazeSearchProblem </b>  
Esta es una subclase de SearchProblem que implementa concretamente el problema de busqueda en laberinto. El constructor recibe el laberinto en un objeto maze. Cada estado es codificado como una tupla (row,col) representando la posicion de una celda del grid. El estado inicial es la celda de inicio y el único estado objetivo es la celda de salida.
Se necesita completar Actions (acciones legales para un estado dado) y result (que hacen las acciones).

In [0]:
class MazeSearchProblem(SearchProblem):
    def __init__(self, maze):
        """El constructor recibe el maze"""
        self.maze = maze
        self.initial = maze.getStartCell()
        self.goal = maze.getExitCell()
        self.numNodesExpanded = 0        
        self.expandedNodeSet = {}   
        
    def __isValidState(self,state):
        """ Retorna true si el estado dado corresponde a una celda no bloqueada valida """
        row,col = state
        if row < 0 or row >= self.maze.getNumRows():
            return False
        if col < 0 or col >= self.maze.getNumCols():
            return False
        return not self.maze.isBlocked(row,col)        

    def actions(self, state):
        """Retorna las acciones legales desde la celda actual """
        row,col = state
        acciones = []
        if self.__isValidState((row-1, col)):
            acciones.append('N')

        ## TO DO: Completar    

        ##################       

        return acciones
    
    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada desde la celda actual.
        La accion debe ser alguna de self.actions(state)"""  
        row,col = state
        newState = ()
        if action == 'N':
            newState = (row-1, col)
        ## TO DO: Completar 

        ##################      
        return newState
        
    def goal_test(self, state):
        """Retorna True si state es self.goal"""
        return (self.goal == state) 

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con la accion action 
        El costo del camino para llegar a state1 es c. El costo de la accion sale de self.maze """
        row, col = state2
        actionCost = None

        ## TO DO: Completar
        ## Defina el valor de la variable actionCost, de acuerdo al tipo de celda (con agua '~', 
        ## con arena '=' o vacía ' '). Por ejemplo, si es una celda con agua, el costo de la accion
        ## es 5. Para esta parte de la implementación, no se requiere verificar que la celda
        ## sea una celda con obstáculo ('#').
        if maze.grid[row][col] == ' ':
            actionCost = 1 #si es una celda vacía, el costo de la accion es 1
        elif maze.grid[row][col] == '~':
            actionCost = 5 #si es una celda con agua, el costo de la accion es 5
        elif maze.grid[row][col] == '=':
            actionCost = 3 #si es una celda con arena, el costo de la accion es 3
        ##################
        
        if state2 == self.maze.getStartCell() or state2 == self.maze.getExitCell():
            actionCost = 1
        
        return c + actionCost

### <b> Funcion para mostrar los resultados </b> 

In [0]:
def displayResults(maze, visitedNodes, solutionNodes):
    """ Muestra los resultados de busqueda en el maze.   """
    grid_copy = []
    for row in maze.grid:
        grid_copy.append([x for x in row]) 
    for node in visitedNodes:
        row,col = node.state
        ch = maze.grid[row][col]
        if ch != 'S' and ch != 'E': grid_copy[row][col] = 'o' 
    for node in solutionNodes:  
        row,col = node.state
        ch = maze.grid[row][col]
        if ch != 'S' and ch != 'E': grid_copy[row][col] = 'x'    
    maze_copy = Maze(grid_copy)
    print (maze_copy)
    print ("x - celdas en la solucion")
    print ("o - celdas visitadas durante la búsqueda")
    print ("-------------------------------")

## <b> Experimentación con los algoritmos de Busqueda</b> 


In [0]:
""" Carga un laberinto de archivo en disco e instancia el problema de busqueda.   """
maze = readMazeFromFile('mapa_sesion2.txt') 
p = MazeSearchProblem(maze)
print(maze)

 --------------------------- 
|#################~~~~~~~~~~|
|#################    =~   #|
|     ~~~~    ==   =~==~   #|
|     ~~S~~  ====  ==~=~~  #|
|     ===~~   #==   =~=~~  #|
|     ===~~  ###    =~     #|
|     ===~~  ##   ###~     #|
| =~              ## ~   ###|
| =~      ##  == =~ =~   ###|
|      ==###  == =~      ###|
|##########   ==        ~###|
|~~~~~~~~         ~~~~##~   |
| ### # # ### ### ====##=   |
| # # # # #   # #   ==~~~=  |
| ### # # #   ###   =~~~==  |
| #   # # #   #     =##~E   |
| #   ### ### #     =##     |
|~~~~~~~~~~~~~~~~~~~~~~~~~~~|
 --------------------------- 


### busqueda en amplitud (BFS)

In [0]:
nsol, visited_nodes = graph_search(p, FIFOQueue())
print('Solucion BFS: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion BFS: ['S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'E']. Nodos visitados=339. Costo Solucion = 52
 --------------------------- 
|#################oooooooooo|
|#################ooooooooo#|
|oooooooooooooooooooooooooo#|
|oooooooSoooooooooooooooooo#|
|oooooooxooooo#oooooooooooo#|
|oooooooxoooo###ooooooooooo#|
|oooooooxoooo##ooo###oooooo#|
|oooooooxxxxxooooo##ooooo###|
|ooooooooo##xoooooooooooo###|
|oooooooo###xoooooooooooo###|
|##########oxoooooooooooo###|
|oooooooooooxxxxxxoooo##oooo|
|o###o#o#o###o###xoooo##ooo |
|o# #o#o#o#ooo# #xoooooooo  |
|o###o#o#o#ooo###xxxxxxxo=  |
|o#ooo#o#o#ooo#oooooo##xE   |
|o#ooo###o###o#oooooo##o    |
|~ooooooooooooooooooooo~~~~~|
 --------------------------- 
x - celdas en la solucion
o - celdas visitadas durante la búsqueda
-------------------------------


### busqueda en profundidad (DFS)

In [0]:
nsol, visited_nodes = graph_search(p, [])
print('Solucion DFS: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion DFS: ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'S', 'W', 'S', 'W', 'W', 'W', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'S', 'S', 'E', 'E', 'E', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'N', 'N', 'W', 'W', 'W']. Nodos visitados=158. Costo Solucion = 198
 --------------------------- 
|#################~~~~~~~~~~|
|#################    =~   #|
|ooooooo~~    ==   =~==~   #|
|oooooo~Sxxxxxxxxxxxxxxxxxx#|
|ooooooo=~~   #==   =~=~~ x#|
|oooooooooooo###xxxxxxxxxxx#|
|ooooooooooo ##xx ###~     #|
|oooooooooooxxxx  ## ~   ###|
|ooooooooo##x == =~ =~   ###|
|oooooooo###xxxxxxxxxxxxx###|
|##########   ==        x###|
|~~~~~~~~         ~~~~##xxxx|
| ### # # ### ### ====##=  x|
| # # # # #   # #xxxxxxxxxxx|
| ### # # #   ###x  =~~~==  |
| #   # # #   #  xxxx##~Exxx|
| 

### busqueda A* con heurística nula (UCS)

In [0]:
nsol, visited_nodes = astar_search(p, nullheuristic)
print('Solucion A* y heuristica nula (UCS): {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion A* y heuristica nula (UCS): ['S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'E', 'E', 'S', 'S', 'S', 'S', 'W', 'W']. Nodos visitados=327. Costo Solucion = 46
 --------------------------- 
|#################ooooooooo~|
|#################ooooooooo#|
|oooooooooooooooooooooooooo#|
|oooooooSoooooooooooooooooo#|
|oooooooxooooo#oooooooooooo#|
|oooooooxoooo###ooooooooooo#|
|oooooooxoooo##ooo###oooooo#|
|oooooooxxxxxxxxxo##ooooo###|
|ooooooooo##ooooxoooooooo###|
|oooooooo###ooooxoooooooo###|
|##########oooooxxxxxxxxx###|
|~~~oooooooooooooooooo##xxxo|
| ###o#o#o###o###ooooo##ooxo|
| # #o#o#o#ooo# #oooooooooxo|
| ###o#o#o#ooo###ooooooo=oxo|
| # oo#o#o#ooo#oooooo##~Exxo|
| #   ###o###o#oooooo##   o |
|~~~~~~ooooooooooooooo~~~~~~|
 --------------------------- 
x - celdas en la solucion
o - celdas visitadas durante la búsqueda
-------------------------------


### busqueda A* con heurística straightline_dist 

In [0]:
nsol, visited_nodes = astar_search(p, straightline_dist)
print('Solucion A* y heuristica straightline_dist: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion A* y heuristica straightline_dist: ['S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'E', 'S', 'E', 'S', 'S', 'S', 'W', 'W']. Nodos visitados=271. Costo Solucion = 46
 --------------------------- 
|#################oo~~~~~~~~|
|#################ooooo~   #|
|ooooooooooooooooooooo=~   #|
|oooooooSoooooooooooo~=~~  #|
|oooooooxooooo#ooooooo=~~  #|
|oooooooxoooo###oooooooooo #|
|oooooooxoooo##ooo###oooooo#|
|oooooooxxxxxxxxxo##ooooo###|
|ooooooooo##ooooxoooooooo###|
|oooooooo###ooooxoooooooo###|
|##########oooooxxxxxxxxx###|
|~~~~~~~oooooooooooooo##xxoo|
| ### # #o###o###ooooo##oxx |
| # # # #o#ooo# #ooooooo~ox |
| ### # #o#ooo###oooooo~==x |
| #   # #o#ooo#oooooo##~Exx |
| #   ###o###o#oooooo##     |
|~~~~~~~~~~~~oooooooo~~~~~~~|
 --------------------------- 
x - celdas en la solucion
o - celdas visitadas durante la búsqueda
-------------------------------


### busqueda A* con heurística manhatan_dist 

In [0]:
nsol, visited_nodes = astar_search(p, manhatan_dist)
print('Solucion A* y heuristica manhatan_dist: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion A* y heuristica manhatan_dist: ['S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'E', 'S', 'E', 'S', 'S', 'S', 'W', 'W']. Nodos visitados=249. Costo Solucion = 46
 --------------------------- 
|#################~~~~~~~~~~|
|#################ooooo~   #|
|   oooooooooooooooo~o=~   #|
|  oooooSoooooooooooo~=~~  #|
| ooooooxooooo#ooooooo=~~  #|
| ooooooxoooo###oooooooooo #|
| ooooooxoooo##ooo###~ooooo#|
| =~ooooxxxxxxxxxo##ooooo###|
| =~oooooo##ooooxoooooooo###|
| ooooooo###ooooxoooooooo###|
|##########oooooxxxxxxxxx###|
|~~~~~~~oooooooooooooo##xxo |
| ### # #o###o###ooooo##oxx |
| # # # #o#ooo# #ooooooo~ox |
| ### # #o#ooo###oooooo~==x |
| #   # #o#ooo#oooooo##~Exx |
| #   ###o###o#oooooo##     |
|~~~~~~~~~~~~o~~ooooo~~~~~~~|
 --------------------------- 
x - celdas en la solucion
o - celdas visitadas durante la búsqueda
-------------------------------


## Preguntas

 
<b>1) De acuerdo a los resultados encontrados, ¿los algoritmos BFS y DFS encuentran soluciones optimas? ¿Por qué?.  </b>

<b>2) En cuanto a las soluciones encontradas por A* utilizando las dos heurísticas (por separado), siempre se encontrarán soluciones óptimas? ¿Por qué?. </b>

<b>3) ¿Cuál método visita menos nodos? ¿Por qué?. </b>

<b>4) Existe relación de dominancia entre las dos heurísticas implementadas? ¿Por qué? ¿Cómo las puede comparar?. </b>





 